In [46]:
import os
import shutil
import requests

class Dota2HeroPickImageDownloader:
    def __init__(self, img_folder='hero_pick_images'):
        self.base_url = 'https://cdn.akamai.steamstatic.com/apps/dota2/images/dota_react/heroes/'
        self.img_folder = img_folder
        self.heroes = [
    "abaddon",
    "alchemist",
    "ancient_apparition",
    "antimage",
    "arc_warden",
    "axe",
    "bane",
    "batrider",
    "beastmaster",
    "bloodseeker",
    "bounty_hunter",
    "brewmaster",
    "bristleback",
    "broodmother",
    "centaur",
    "chaos_knight",
    "chen",
    "clinkz",
    "rattletrap",
    "crystal_maiden",
    "dark_seer",
    "dark_willow",
    "dawnbreaker",
    "dazzle",
    "death_prophet",
    "disruptor",
    "doom_bringer",
    "dragon_knight",
    "drow_ranger",
    "earth_spirit",
    "earthshaker",
    "elder_titan",
    "ember_spirit",
    "enchantress",
    "enigma",
    "faceless_void",
    "primal_beast",
    "grimstroke",
    "gyrocopter",
    "hoodwink",
    "huskar",
    "invoker",
    "wisp",
    "jakiro",
    "juggernaut",
    "keeper_of_the_light",
    "kunkka",
    "legion_commander",
    "leshrac",
    "lich",
    "life_stealer",
    "lina",
    "lion",
    "lone_druid",
    "luna",
    "lycan",
    "magnataur",
    "marci",
    "mars",
    "medusa",
    "meepo",
    "ringmaster",
    "mirana",
    "monkey_king",
    "morphling",
    "muerta",
    "naga_siren",
    "furion",
    "necrolyte",
    "night_stalker",
    "nyx_assassin",
    "ogre_magi",
    "omniknight",
    "oracle",
    "obsidian_destroyer",
    "pangolier",
    "phantom_assassin",
    "phantom_lancer",
    "phoenix",
    "puck",
    "pudge",
    "pugna",
    "queenofpain",
    "razor",
    "riki",
    "rubick",
    "sand_king",
    "shadow_demon",
    "nevermore",
    "shadow_shaman",
    "silencer",
    "skywrath_mage",
    "slardar",
    "slark",
    "snapfire",
    "sniper",
    "spectre",
    "spirit_breaker",
    "storm_spirit",
    "sven",
    "techies",
    "templar_assassin",
    "terrorblade",
    "tidehunter",
    "shredder",
    "tinker",
    "tiny",
    "treant",
    "troll_warlord",
    "tusk",
    "abyssal_underlord",
    "undying",
    "ursa",
    "vengefulspirit",
    "venomancer",
    "viper",
    "visage",
    "void_spirit",
    "warlock",
    "weaver",
    "windrunner",
    "winter_wyvern",
    "witch_doctor",
    "skeleton_king",
    "zuus"
]
    
    def download_file(self, hero):
        local_filename = hero + '.png'
        path = os.path.join("{}/{}".format(self.img_folder, local_filename))
        with requests.get(self.base_url + local_filename, stream=True) as r:
            with open(path, 'wb') as f:
                shutil.copyfileobj(r.raw, f)

        return local_filename
    
    def download_pick_hero_images(self):
        for hero in self.heroes:
            self.download_file(hero)
    
#Baixa todas as imagens e salva na pasta
image_downloader = Dota2HeroPickImageDownloader()
image_downloader.download_pick_hero_images()

In [109]:
import cv2
import numpy as np
from pathlib import Path
import os

class Dota2ImageGenerator:
    def __init__(self, source_dir='hero_images', output_dir='hero_dataset'):
        self.source_dir = Path(source_dir)
        self.output_dir = Path(output_dir)
        
        # Criar diretórios de saída se não existirem
        self.output_dir.mkdir(exist_ok=True)
    
    def apply_ban_effect(self, image):
        """Aplica efeito de banimento na imagem do herói"""
        height, width = image.shape[:2]
        
        # Criar máscara escura
        darkened = cv2.addWeighted(image, 0.6, np.zeros_like(image), 0.4, 0)
        
        # Criar linha diagonal vermelha
        mask = np.zeros_like(image)
        cv2.line(mask, (width, 0), (0, height), (0, 0, 255), thickness=3)
        
        # Sobrepor a linha na imagem escurecida
        result = cv2.addWeighted(darkened, 1, mask, 0.7, 0)
        
        return result
    
    def generate_variations(self, hero_name, image, ban = False):
        """Gera variações da imagem do herói"""
        
        if (ban):
            image = self.apply_ban_effect(image)
        
        state = 'ban' if ban else 'pick'
        
        hero_state_dir = self.output_dir / f"{hero_name}_{state}"
        hero_state_dir.mkdir(exist_ok=True)

        # Salvar imagem base
        cv2.imwrite(str(hero_state_dir / "base.png"), image)

#         # Gerar variações com ruído e diferentes níveis de brilho
#         for i in range(5):
#             # Adicionar ruído gaussiano
#             noise = np.random.normal(0, 5, image.shape).astype(np.uint8)
#             noisy = cv2.add(image, noise)

#             # Variar brilho
#             brightness = 0.8 + (i * 0.1)  # 0.8 a 1.2
#             bright = cv2.convertScaleAbs(image, alpha=brightness, beta=0)

#             # Salvar variações
#             cv2.imwrite(str(hero_state_dir / f"noisy_{i}.png"), noisy)
#             cv2.imwrite(str(hero_state_dir / f"bright_{i}.png"), bright)
    
    def process_heroes(self, source_dir, ban):
        """Processa todas as imagens de heróis no diretório fonte"""
        for img_path in Path(source_dir).glob('*.png'):
            hero_name = img_path.stem
            print(f"Processando herói: {hero_name}")
            
            # Ler imagem
            image = cv2.imread(str(img_path))
            if image is None:
                print(f"Erro ao ler imagem: {img_path}")
                continue
            
            # Gerar variações
            self.generate_variations(hero_name, image, ban)
            
    def process_pick_heroes(self, source_dir = 'hero_pick_images'):
        self.process_heroes(source_dir, False)
        
    def process_ban_heroes(self, source_dir = 'hero_ban_images'):
        self.process_heroes(source_dir, True)

def prepare_dataset():
    """Prepara o dataset completo com classes separadas"""
    # Configurar gerador
    generator = Dota2ImageGenerator()
    
    # Processar imagens de picks
    generator.process_pick_heroes('hero_pick_images')
    # Processar imagens de bans
    generator.process_ban_heroes('hero_ban_images')

In [110]:
#Cria variações de imagens a partir da base de imagens
prepare_dataset()

Processando herói: bloodseeker
Processando herói: jakiro
Processando herói: elder_titan
Processando herói: gyrocopter
Processando herói: earth_spirit
Processando herói: ancient_apparition
Processando herói: enchantress
Processando herói: sand_king
Processando herói: windrunner
Processando herói: pudge
Processando herói: slardar
Processando herói: marci
Processando herói: lone_druid
Processando herói: weaver
Processando herói: batrider
Processando herói: huskar
Processando herói: silencer
Processando herói: chen
Processando herói: bristleback
Processando herói: lycan
Processando herói: faceless_void
Processando herói: zuus
Processando herói: furion
Processando herói: luna
Processando herói: spirit_breaker
Processando herói: omniknight
Processando herói: antimage
Processando herói: alchemist
Processando herói: tinker
Processando herói: necrolyte
Processando herói: rubick
Processando herói: razor


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Processando herói: ursa
Processando herói: arc_warden
Processando herói: clinkz
Processando herói: bane
Processando herói: nyx_assassin
Processando herói: techies
Processando herói: pangolier
Processando herói: templar_assassin
Processando herói: skeleton_king
Processando herói: tiny
Processando herói: terrorblade
Processando herói: shadow_demon
Processando herói: abyssal_underlord
Processando herói: crystal_maiden
Processando herói: shadow_shaman
Processando herói: phantom_assassin
Processando herói: treant
Processando herói: oracle
Processando herói: spectre
Processando herói: juggernaut
Processando herói: dark_seer
Processando herói: tusk
Processando herói: lich
Processando herói: death_prophet
Processando herói: skywrath_mage
Processando herói: lion
Processando herói: magnataur
Processando herói: wisp
Processando herói: grimstroke
Processando herói: viper
Processando herói: phantom_lancer
Processando herói: hoodwink
Processando herói: meepo
Processando herói: ringmaster
Processando

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Processando herói: witch_doctor
Processando herói: pugna
Processando herói: axe
Processando herói: ember_spirit
Processando herói: storm_spirit
Processando herói: sven
Processando herói: obsidian_destroyer
Processando herói: riki
Processando herói: puck
Processando herói: abaddon
Processando herói: broodmother
Processando herói: nevermore
Processando herói: bounty_hunter
Processando herói: venomancer
Processando herói: medusa
Processando herói: monkey_king
Processando herói: night_stalker
Processando herói: morphling
Processando herói: shredder
Processando herói: keeper_of_the_light
Processando herói: undying
Processando herói: life_stealer
Processando herói: mars
Processando herói: primal_beast
Processando herói: enigma
Processando herói: disruptor
Processando herói: queenofpain
Processando herói: void_spirit
Processando herói: earthshaker
Processando herói: brewmaster
Processando herói: rattletrap
Processando herói: kunkka
Processando herói: muerta
Processando herói: ogre_magi
Proces

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Processando herói: centaur
Processando herói: beastmaster
Processando herói: legion_commander
Processando herói: dazzle
Processando herói: drow_ranger
Processando herói: invoker
Processando herói: troll_warlord
Processando herói: snapfire
Processando herói: leshrac
Processando herói: vengefulspirit
Processando herói: naga_siren
Processando herói: visage
Processando herói: sniper
Processando herói: dawnbreaker
Processando herói: phoenix
Processando herói: windrunner
Processando herói: sniper


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [114]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
#import pyautogui
import time
from PIL import Image
import os
from pathlib import Path
from datetime import datetime, timezone

class Dota2Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        
        # Encontrar todos os heróis e seus estados (pick/ban)
        self.classes = []
        for path in self.data_dir.glob("*_*"):
            if path.is_dir():
                self.classes.append(path.name)  # Formato: "heroname_state"
        
        self.classes = sorted(self.classes)
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        
        self.images = []
        self.labels = []
        
        # Carregar imagens e labels
        for class_name in self.classes:
            class_dir = self.data_dir / class_name
            for img_path in class_dir.glob('*.png'):
                self.images.append(str(img_path))
                self.labels.append(self.class_to_idx[class_name])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        return image, label

class Dota2HeroClassifier:
    def __init__(self, model_path=None, data_dir='hero_dataset'):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Definir transformações para as imagens
        self.transform = transforms.Compose([
            transforms.Resize((256, 144)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        # Carregar dataset e criar modelo
        self.dataset = Dota2Dataset(data_dir, self.transform)
        
        self.model = self._create_model()
        
        if model_path and os.path.exists(model_path):
            self.model.load_state_dict(torch.load(model_path, map_location=self.device))
            print(f"Modelo carregado de {model_path}")
            
        self.model = self.model.to(self.device)
        
        # Criar mapeamento reverso de índice para classe
        self.idx_to_class = {v: k for k, v in self.dataset.class_to_idx.items()}
    
    def _create_model(self):
        """Cria um modelo MobileNetV2 pré-treinado e modifica a última camada"""
        model = models.mobilenet_v2(pretrained=True)
        
        # Congelar os pesos do modelo base
        for param in model.parameters():
            param.requires_grad = False
            
        # Modificar a última camada para o número de classes
        num_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_features, len(self.dataset.classes))
        
        return model
    
    def train(self, epochs=10, batch_size=16, learning_rate=0.00001):
        """Treina o modelo com o dataset fornecido"""
        self.model.train()
        
        train_loader = DataLoader(self.dataset, batch_size=batch_size, shuffle=True)
        optimizer = optim.Adam(self.model.classifier.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            
            for images, labels in train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            
            epoch_loss = running_loss / len(train_loader)
            accuracy = 100 * correct / total
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')
    
#     def predict_image(self, image):
#         """Prediz o herói e seu estado em uma imagem"""
#         self.model.eval()
        
#         # Converter imagem para PIL se necessário
#         if isinstance(image, np.ndarray):
#             image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
#         # Aplicar transformações
#         image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        
#         with torch.no_grad():
#             outputs = self.model(image_tensor)
#             _, predicted = torch.max(outputs, 10)
#             confidence = torch.nn.functional.softmax(outputs, dim=1).max().item()
#             print(f'Confidence {confidence}')
#             if confidence > 0.8:  # Ajuste este limiar conforme necessário
#                 class_name = self.idx_to_class[predicted.item()]
#                 hero_name, state = class_name.split('_')
#                 return {'hero': hero_name, 'state': state, 'confidence': confidence}
#             return None

    def predict_image(self, image, top_k=8):
        """Prediz os heróis e seus estados em uma imagem, retornando os top_k resultados"""
        self.model.eval()

        # Converter imagem para PIL se necessário
        if isinstance(image, np.ndarray):
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Aplicar transformações
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            outputs = self.model(image_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            top_probs, top_indices = torch.topk(probabilities, top_k, dim=1)

            predictions = []
            for i in range(top_k):
                confidence = top_probs[0, i].item()
                class_idx = top_indices[0, i].item()
                class_name = self.idx_to_class[class_idx]
                
                state = class_name.split('_')[-1]
                
                hero_name = class_name[:len(class_name)-len(state)-1]
                predictions.append({'hero': hero_name, 'state': state, 'confidence': confidence})

            return predictions
    
    def analyze_draft_image(self, image_path):
        """Analisa uma imagem estática da tela de draft"""
        # Carregar imagem
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
            
        height, width = image.shape[:2]
        mid_width = width // 2
        pick_height = int(height * 0.1)
        
        # Dividir a imagem em lado radiant (esquerdo) e dire (direito)
        radiant_side = image[:pick_height, :mid_width]
        dire_side = image[:pick_height, mid_width:]
        
        results = {
            'radiant': {'picks': [], 'bans': []},
            'dire': {'picks': [], 'bans': []}
        }
        
        # Analisar lado radiant
        predictions = self.process_team_side(radiant_side)
        for pred in predictions:
            if pred['state'] == 'pick':
                results['radiant']['picks'].append(pred['hero'])
            else:
                results['radiant']['bans'].append(pred['hero'])
        
        # Analisar lado dire
        predictions = self.process_team_side(dire_side)
        for pred in predictions:
            if pred['state'] == 'pick':
                results['dire']['picks'].append(pred['hero'])
            else:
                results['dire']['bans'].append(pred['hero'])
        
        return results
    
    def process_team_side(self, image):
        """Processa um lado da imagem (radiant ou dire) para encontrar heróis"""
        # Obtém o timestamp atual em UTC
        timestamp = datetime.now(timezone.utc).timestamp()
        # Salvar a imagem
        cv2.imwrite(str(timestamp) + '.png', image)
        
        predictions = []
        height, width = image.shape[:2]
        
        result = self.predict_image(image)
        print(result)
        
        return predictions
        
#         # Você precisará ajustar estes parâmetros baseado no seu layout
#         regions = []
        
#         # Exemplo de regiões para picks (ajuste conforme necessário)
#         pick_y = height // 4
#         for i in range(5):  # 5 picks por time
#             x = (width // 6) * (i + 1)
#             regions.append((x - 50, pick_y - 50, 100, 100))  # (x, y, width, height)
        
#         # Exemplo de regiões para bans (ajuste conforme necessário)
#         ban_y = height * 3 // 4
#         for i in range(7):  # 7 bans por time
#             x = (width // 8) * (i + 1)
#             regions.append((x - 50, ban_y - 50, 100, 100))
        
#         # Analisar cada região
#         for x, y, w, h in regions:
#             roi = image[y:y+h, x:x+w]
#             prediction = self.predict_image(roi)
#             if prediction:
#                 predictions.append(prediction)
        
#         return predictions
    
#     def run_live(self, interval=0.5):
#         """Executa o classificador em tempo real"""
#         try:
#             while True:
#                 screenshot = np.array(pyautogui.screenshot())
#                 results = self.analyze_draft_image(screenshot)
                
#                 print("\nStatus atual:")
#                 print("Radiant picks:", results['radiant']['picks'])
#                 print("Radiant bans:", results['radiant']['bans'])
#                 print("Dire picks:", results['dire']['picks'])
#                 print("Dire bans:", results['dire']['bans'])
                
#                 time.sleep(interval)
#         except KeyboardInterrupt:
#             print("\nMonitoramento encerrado pelo usuário.")
    
    def save_model(self, path='dota2_model.pth'):
        """Salva o modelo treinado"""
        torch.save(self.model.state_dict(), path)
        print(f"Modelo salvo em {path}")

# Função auxiliar para teste
def test_draft_image(model_path, image_path):
    """Testa o modelo com uma imagem estática de draft"""
    classifier = Dota2HeroClassifier(model_path=model_path)
    results = classifier.analyze_draft_image(image_path)
    
    print("\nResultados da análise:")
    print("\nRadiant:")
    print(f"Picks: {results['radiant']['picks']}")
    print(f"Bans: {results['radiant']['bans']}")
    print("\nDire:")
    print(f"Picks: {results['dire']['picks']}")
    print(f"Bans: {results['dire']['bans']}")
    
    return results

In [116]:
classifier = Dota2HeroClassifier()
classifier.train(epochs=100)
classifier.save_model('dota2_model.pth')

Using device: cuda
Epoch [1/100], Loss: 4.9423, Accuracy: 0.00%
Epoch [2/100], Loss: 4.9442, Accuracy: 0.79%
Epoch [3/100], Loss: 4.9326, Accuracy: 2.36%
Epoch [4/100], Loss: 4.9276, Accuracy: 0.79%
Epoch [5/100], Loss: 4.8919, Accuracy: 0.79%
Epoch [6/100], Loss: 4.8794, Accuracy: 0.79%
Epoch [7/100], Loss: 4.8906, Accuracy: 3.15%
Epoch [8/100], Loss: 4.8533, Accuracy: 2.36%
Epoch [9/100], Loss: 4.8616, Accuracy: 2.36%
Epoch [10/100], Loss: 4.8623, Accuracy: 1.57%
Epoch [11/100], Loss: 4.8096, Accuracy: 2.36%
Epoch [12/100], Loss: 4.7980, Accuracy: 3.94%
Epoch [13/100], Loss: 4.7919, Accuracy: 2.36%
Epoch [14/100], Loss: 4.7571, Accuracy: 2.36%
Epoch [15/100], Loss: 4.7941, Accuracy: 0.79%
Epoch [16/100], Loss: 4.7746, Accuracy: 2.36%
Epoch [17/100], Loss: 4.7166, Accuracy: 2.36%
Epoch [18/100], Loss: 4.7248, Accuracy: 4.72%
Epoch [19/100], Loss: 4.6913, Accuracy: 4.72%
Epoch [20/100], Loss: 4.7406, Accuracy: 4.72%
Epoch [21/100], Loss: 4.7036, Accuracy: 3.15%
Epoch [22/100], Loss: 4.

In [117]:
test_draft_image('dota2_model.pth', 'DOTA_2_DRAFT.png')

Using device: cuda
Modelo carregado de dota2_model.pth
[{'hero': 'hoodwink', 'state': 'pick', 'confidence': 0.019447699189186096}, {'hero': 'tinker', 'state': 'pick', 'confidence': 0.018925413489341736}, {'hero': 'dragon_knight', 'state': 'pick', 'confidence': 0.01808910258114338}, {'hero': 'oracle', 'state': 'pick', 'confidence': 0.014532321132719517}, {'hero': 'lina', 'state': 'pick', 'confidence': 0.014462126418948174}, {'hero': 'dazzle', 'state': 'pick', 'confidence': 0.014338369481265545}, {'hero': 'skywrath_mage', 'state': 'pick', 'confidence': 0.014156664721667767}, {'hero': 'pangolier', 'state': 'pick', 'confidence': 0.013881517574191093}]
[{'hero': 'dragon_knight', 'state': 'pick', 'confidence': 0.018636172637343407}, {'hero': 'crystal_maiden', 'state': 'pick', 'confidence': 0.016448231413960457}, {'hero': 'lina', 'state': 'pick', 'confidence': 0.015165689401328564}, {'hero': 'hoodwink', 'state': 'pick', 'confidence': 0.015005847439169884}, {'hero': 'pangolier', 'state': 'pick

{'radiant': {'picks': [], 'bans': []}, 'dire': {'picks': [], 'bans': []}}

In [69]:
transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])

dataset = Dota2Dataset('hero_dataset', transform)

In [99]:
dataset.class_to_idx

{'abaddon_pick': 0,
 'abyssal_underlord_pick': 1,
 'alchemist_pick': 2,
 'ancient_apparition_pick': 3,
 'antimage_pick': 4,
 'arc_warden_pick': 5,
 'axe_pick': 6,
 'bane_pick': 7,
 'batrider_pick': 8,
 'beastmaster_pick': 9,
 'bloodseeker_pick': 10,
 'bounty_hunter_pick': 11,
 'brewmaster_pick': 12,
 'bristleback_pick': 13,
 'broodmother_pick': 14,
 'centaur_pick': 15,
 'chaos_knight_pick': 16,
 'chen_pick': 17,
 'clinkz_pick': 18,
 'crystal_maiden_pick': 19,
 'dark_seer_pick': 20,
 'dark_willow_pick': 21,
 'dawnbreaker_pick': 22,
 'dazzle_pick': 23,
 'death_prophet_pick': 24,
 'disruptor_pick': 25,
 'doom_bringer_pick': 26,
 'dragon_knight_pick': 27,
 'drow_ranger_pick': 28,
 'earth_spirit_pick': 29,
 'earthshaker_pick': 30,
 'elder_titan_pick': 31,
 'ember_spirit_pick': 32,
 'enchantress_pick': 33,
 'enigma_pick': 34,
 'faceless_void_pick': 35,
 'furion_pick': 36,
 'grimstroke_pick': 37,
 'gyrocopter_pick': 38,
 'hoodwink_pick': 39,
 'huskar_pick': 40,
 'invoker_pick': 41,
 'jakiro_p